# Import Libraries

# Step 3:

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
cust = pd.read_csv(r'/Users/shailioza/Documents/CareerFoundry Projects/Instacart Basket Analysis/02 Data/Original Data/customers.csv', index_col = False)

# Step 4:

In [3]:
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [4]:
cust.shape

(206209, 10)

In [5]:
# The names of the customers may not be required for the analysis as we already have user ids. However, I am deciding not to remove them from the dataframe right now

# I would like to change the column name 'Surnam' to 'Last Name' to avoid any confusion.

cust.rename(columns = {'Surnam': 'Last Name'}, inplace = True)

In [6]:
cust.head()

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# Step 5:

In [7]:
cust.dtypes

user_id          int64
First Name      object
Last Name       object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [8]:
cust['user_id'] = cust['user_id'].astype('str')

In [9]:
print(cust['user_id'].dtypes)

object


In [10]:
cust.describe()

,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


In [11]:
# Check for missing values

cust.isnull().sum()

user_id             0
First Name      11259
Last Name           0
Gender              0
STATE               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

There is missing data in the first name column, but since that column is not necessary for analysis, we will not do anything to change it.

In [12]:
# Check for mixed data

for col in cust.columns.tolist():
    weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis=1)
    if len (cust[weird]) > 0:
        print (col)

First Name


The 'First Name' column is shown as mixed type because it has some Nan values. Changing the column data type to string will remove that inconsistency.

In [13]:
cust['First Name'] = cust['First Name'].astype('str')

In [14]:
# Rerunning the code for mixed data types

for col in cust.columns.tolist():
    weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis=1)
    if len (cust[weird]) > 0:
        print (col)

In [15]:
# Checking for duplicates

df_dups = cust[cust.duplicated()]

In [16]:
df_dups 

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


There are no duplicates in the original dataframe.

# Step 6:

In [17]:
df_merged = pd.read_pickle('/Users/shailioza/Documents/CareerFoundry Projects/Instacart Basket Analysis/02 Data/Prepared Data/orders_products_merged_aggregated.pkl')

In [18]:
df_merged.head()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,busiest_day,busiest_days,busiest_period_of_day,price_range_loc,max_order,loyalty_flag,avg_price,spending_flag,median_freq,order_freq_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Regularly busy,Regularly days,Average orders,Mid-range product,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,1,2398795,1,2,3,7,15.0,196,1,1,...,Regularly busy,Slowest days,Average orders,Mid-range product,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2,473747,1,3,3,12,21.0,196,1,1,...,Regularly busy,Slowest days,Most orders,Mid-range product,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,3,2254736,1,4,4,7,29.0,196,1,1,...,Least busy,Slowest days,Average orders,Mid-range product,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,4,431534,1,5,4,15,28.0,196,1,1,...,Least busy,Slowest days,Most orders,Mid-range product,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


Merging the dataframes by 'user_id' would be ideal in this case.

In [19]:
df_merged['user_id'] = df_merged['user_id'].astype('str')

In [20]:
df_merged.dtypes

Unnamed: 0                   int64
order_id                     int64
user_id                     object
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
price_range_loc             object
max_order                    int64
loyalty_flag                object
avg_price                  float64
spending_flag               object
median_freq                float64
order_freq_flag             object
dtype: object

In [21]:
df_combined = df_merged.merge(cust, on = 'user_id', indicator = 'True')

In [22]:
df_combined.head()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,True
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,1,2398795,1,2,3,7,15.0,196,1,1,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2,473747,1,3,3,12,21.0,196,1,1,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,3,2254736,1,4,4,7,29.0,196,1,1,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,4,431534,1,5,4,15,28.0,196,1,1,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


# Step 8

In [23]:
# Exporting the dataframe

df_combined.to_pickle(r'/Users/shailioza/Documents/CareerFoundry Projects/Instacart Basket Analysis/02 Data/Prepared Data/customers_merged.pkl')